In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir -p /content/drive/MyDrive/ASR/data

In [3]:
%cd /content/drive/MyDrive/ASR/data/

/content/drive/.shortcut-targets-by-id/1378P6eXj44u000tRle-P5AK3e_KHwlv4/ASR/data


In [4]:
# !tar -xzf train-clean-100.tar.gz
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (344 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
#Checking Data structure
!tree -L 3 /content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/ | head -30

#.flac count
!find /content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/ -name "*.flac" | wc -l

#Size
!du -sh /content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/

/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/
├── 103
│   ├── 1240
│   │   ├── 103-1240-0000.flac
│   │   ├── 103-1240-0001.flac
│   │   ├── 103-1240-0002.flac
│   │   ├── 103-1240-0003.flac
│   │   ├── 103-1240-0004.flac
│   │   ├── 103-1240-0005.flac
│   │   ├── 103-1240-0006.flac
│   │   ├── 103-1240-0007.flac
│   │   ├── 103-1240-0008.flac
│   │   ├── 103-1240-0009.flac
│   │   ├── 103-1240-0010.flac
│   │   ├── 103-1240-0011.flac
│   │   ├── 103-1240-0012.flac
│   │   ├── 103-1240-0013.flac
│   │   ├── 103-1240-0014.flac
│   │   ├── 103-1240-0015.flac
│   │   ├── 103-1240-0016.flac
│   │   ├── 103-1240-0017.flac
│   │   ├── 103-1240-0018.flac
│   │   ├── 103-1240-0019.flac
│   │   ├── 103-1240-0020.flac
│   │   ├── 103-1240-0021.flac
│   │   ├── 103-1240-0022.flac
│   │   ├── 103-1240-0023.flac
│   │   ├── 103-1240-0024.flac
│   │   ├── 103-1240-0025.flac
│   │   ├── 103-1240-0026.flac
^C
6.2G	/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/


In [6]:
#Install libraries
!pip install -q librosa audiomentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 12.6 MB/s eta 0:00:00


librosa:

Loads audio files (.flac)
Extracts features (mel spectrogram) <br>
Audio effects (time stretch, pitch shift)


audiomentations:

Pre-built audio augmentations (noise, reverb, etc.)


In [7]:
#Testing audio files
import librosa
import soundfile as sf

sample_file = "/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/103/1241/103-1241-0000.flac"
audio, sr = librosa.load(sample_file, sr=16000)
print(f"Audio shape: {audio.shape}")
print(f"Sample rate: {sr}")
print(f"Duration: {len(audio)/sr:.2f} seconds")

# Give transcript prnting
trans_file = "/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/103/1241/103-1241.trans.txt"
with open(trans_file, 'r') as f:
    transcripts = f.readlines()
    print(f"\nSample transcript: {[transcripts[i] for i in range(10)]}")


KeyboardInterrupt: 

In [ ]:
import librosa
import soundfile as sf
import numpy as np
import random
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import Audio

In [ ]:
import os
import random
import librosa
import numpy as np
from datasets import Dataset
from transformers import AutoProcessor, AutoModelForCTC, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import Dict, List, Union
import torch
from scipy.signal import butter, lfilter


Training_dirs = "/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/"
#Training_dirs=kaggle/input/
#--------AUGMENTATION-----------#

class Augmenter:

    def __init__(self,sr=16000,noise_prob=0.4, noise_mean=0,noise_std=0.003,
                 reverb_prob=0.3, reverb_delay=0.025, reverb_decay=0.2,
                 shuffle_prob=0.05, shuffle_segments=3,
                 time_stretch_prob=0.2, time_stretch_range=(0.9, 1.1),
                 gaps_prob=0.08, gaps_n=4, gaps_max_duration=0.1,
                 freq_mask_prob=0.2, freq_mask_n=1,
                 shuffle_seg_dur=0.08, shuffle_overlap=0.02, shuffle_local_range=3):

        self.sr = sr

        self.noise_mean = noise_mean
        self.noise_std  = noise_std
        self.noise_prob = noise_prob
        self.reverb_prob = reverb_prob
        self.reverb_delay = reverb_delay
        self.reverb_decay = reverb_decay
        self.shuffle_prob = shuffle_prob
        self.shuffle_segments = shuffle_segments
        self.time_stretch_prob = time_stretch_prob
        self.time_stretch_range = time_stretch_range
        self.gaps_prob = gaps_prob
        self.gaps_n = gaps_n
        self.gaps_max_duration = gaps_max_duration
        self.freq_mask_prob = freq_mask_prob
        self.freq_mask_n = freq_mask_n
        self.shuffle_seg_dur = shuffle_seg_dur
        self.shuffle_overlap = shuffle_overlap
        self.shuffle_local_range = shuffle_local_range

    def augment(self, audio):
        #Set of distortions to be applied randomly with probabilities below
        distortions = []

        # 1. Noise
        # Adds gaussian noise -> makes it slightly grainy
        # Min max amplitude of noise - not set
        # p=1.0 - always apply noise
        if random.random() < self.noise_prob:
            distortions.append('noise')

        # 2. Reverb
        # Echo effect
        # Delay the audio by 0.1 -> reduce volume -> pad it to original length -> add
        if random.random() < self.reverb_prob:
            distortions.append('reverb')

        # 3. Shuffle
        # Break into n segments and concat them randomly
        if random.random() < self.shuffle_prob:
            distortions.append('shuffle')

        # 4. Time stretch
        # Randomly slows (0.9) or speeds (1.1) the audio / doesn't change pitch
        if random.random() < self.time_stretch_prob:
            distortions.append('time_stretch')

        # 5. Missing Gaps
        # Randomly insert silences/gaps in the audio
        if random.random() < self.gaps_prob:
            distortions.append('missing_gaps')

        # 6. Frequency Masking
        # Randomly masks a range of frequencies in the spectrogram
        #Butterworth filter is better than applying freqeuncy masks on spectogram (which already has frequency bins) because real wrld freq loss occurs during sound capture/transmission, affecting the raw audio.
        #Butterworth simulates this situation by removing frequency content from the waveform which can then go thru the rest of pipeline,
        #additionally it affects the phase relations and harmonics naturally in contrast to the crude zeroing of freq bins in spectogram

        if random.random() < self.freq_mask_prob:
            distortions.append('frequency_masking')



        # Apply selected distortions
        for distortion in distortions:
            if distortion == 'noise':
                audio = self._add_noise(audio)

            elif distortion == 'reverb':
                audio = self._add_reverb(audio)

            elif distortion == 'shuffle':
                audio = self._segment_shuffle(audio)

            elif distortion == 'time_stretch':
                audio = self._time_stretch(audio)

            elif distortion == 'missing_gaps':
                audio = self._add_missing_gaps(audio)

            elif distortion == 'frequency_masking':
                audio = self._add_frequency_mask(audio)

        return audio


    #Augmentation methods
    def _add_noise(self, audio):
        noise = np.random.normal(self.noise_mean, self.noise_std, size=audio.shape[0])
        audio = audio + noise
        return audio

    def _add_reverb(self, audio):
        delay = int(self.reverb_delay * self.sr)  # Delay in samples (0.05 sec)
        reverb = np.pad(audio * self.reverb_decay, (delay, 0)) #Amplitude scaling -> 0.2
        reverb = reverb[:audio.shape[0]]
        return audio + reverb

    def _segment_shuffle(self, audio, n_segments=None):
        # Previous method -> shuffle random large segments / unrealistic and destroys linguistic stuff
        # if n_segments is None:
        #     n_segments = self.shuffle_segments
        # segments = np.array_split(audio, n_segments)
        # np.random.shuffle(segments)
        # return np.concatenate(segments)


        #New method -> try to simulate temporal jitter / noise in the time domain -> split into micro-segments which are overlapping  -> the segments are shuffle locally within shuffle range
        seg_len = int(self.shuffle_seg_dur * self.sr)
        overlap = int(self.shuffle_overlap * self.sr)
        local_range = self.shuffle_local_range
        segments = []
        i = 0
        while i < len(audio):
            end = min(i + seg_len, len(audio))
            segments.append(audio[i:end])
            i += seg_len - overlap
        n_regions = min(4, max(1, len(segments) // 10))
        region_indices = random.sample(range(len(segments)), n_regions)
        shuffled = segments.copy()
        for r in region_indices:
            for offset in range(-local_range, local_range + 1):
                idx = r + offset
                if 0 <= idx < len(segments):
                    shift = random.randint(-local_range, local_range)
                    new_idx = max(0, min(len(segments) - 1, idx + shift))
                    shuffled[idx] = segments[new_idx]
        return np.concatenate(shuffled)[:len(audio)]


    def _time_stretch(self, audio):
          stretched = librosa.effects.time_stretch(audio, rate=random.uniform(*self.time_stretch_range))
          return stretched

    def _add_missing_gaps(self, audio, n_gaps=None, max_gap_duration=None):
        # if n_gaps is None:
        #     n_gaps = self.gaps_n
        # if max_gap_duration is None:
        #     max_gap_duration = self.gaps_max_duration
        # gap_audio = np.copy(audio)
        # for _ in range(n_gaps):
        #     gap_duration = random.uniform(0.1, max_gap_duration)
        #     gap_samples = int(gap_duration * self.sr)
        #     start = random.randint(0, max(1, len(audio) - gap_samples))
        #     gap_audio[start:start + gap_samples] = 0
        # return gap_audio

        #New method -> fill gaps with low level noises and make edges smoother
        if n_gaps is None:
            n_gaps = self.gaps_n
        if max_gap_duration is None:
            max_gap_duration = self.gaps_max_duration

        gap_audio = np.copy(audio)

        for _ in range(n_gaps):
            gap_duration = random.uniform(0.1, max_gap_duration)
            gap_samples = int(gap_duration * self.sr)
            start = random.randint(0, max(1, len(audio) - gap_samples))

            fade_len = min(int(0.05 * gap_samples), gap_samples // 4)
            fade_out = np.linspace(1.0, 0.0, fade_len)
            fade_in = np.linspace(0.0, 1.0, fade_len)

            mid_len = gap_samples - 2 * fade_len

            if mid_len > 0:
                noise = np.random.normal(0, 0.001, size=mid_len)
                gap_audio[start + fade_len:start + fade_len + mid_len] = noise

            gap_audio[start:start + fade_len] *= fade_out
            gap_audio[start + fade_len + mid_len:start + gap_samples] *= fade_in

        return gap_audio[:len(audio)]
    #APPLY FREQUENCY MASKING USING BUTTERWORTH FILTER
    #n_masks -> how many bands will be filtered out
    #A 16 kHz sampler can only capture up to 8 kHz frequencies because you need at least two samples per wave cycle to know what the wave looks like -> nyquist
    def _add_frequency_mask(self, audio, n_masks=None):
      if n_masks is None:
          n_masks = self.freq_mask_n
      masked_audio = audio.copy()
      nyquist = self.sr/2
      for _ in range(n_masks):
        l_freq = random.uniform(500,5000)
        h_freq = l_freq + random.uniform(500,2000)
        l_freq = min(l_freq,nyquist-100)
        h_freq = min(h_freq,nyquist-100)
        b,a = butter(N=4, Wn=[l_freq,h_freq], btype= "bandstop", fs = self.sr) #N=4 th order -> a dip in frequency response where that removal band is with smooth edges
        masked_audio = lfilter(b,a,masked_audio)
      return masked_audio

augmenter = Augmenter()

In [ ]:
def mel_log_gen(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000)
    audio = augmenter.augment(audio)
    sf.write("/content/drive/MyDrive/ASR/data/data_aug"+audio_file[audio_file.rfind('/'):],audio,sr)
    # if isinstance(audio, tf.Tensor):
    #     audio = audio.numpy()
    # mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=80)
    # log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)

tldr; added three kinda custom augmentations to present <br>
frequency mask using butterworth filter <br>
missing gaps with low non 0 noise and fading in nd out <br>
shuffling segments as micro segments with overlap and localized

In [ ]:
#Testing augmentations
sample_file = "/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/6818/68772/6818-68772-0033.flac"
audio,sr = librosa.load(sample_file, sr=16000)
# fig,ax = plt.subplots(1,figsize=(7, 7))
# librosa.display.specshow(mel_log_gen(sample_file), sr=16000, x_axis='time', y_axis='mel', ax=ax,cmap='viridis')
# ax.set_title('Augmented Log-Mel Spectrogram')
# plt.show()

In [ ]:
#Comparison with specaugments freq mask which is applied after spectogram generated
import torch
import torchaudio.transforms as T

audio_check3, sr = librosa.load(sample_file, sr=16000)

mel_og = librosa.feature.melspectrogram(y=audio_check3, sr=sr, n_mels=80)
log_mel_og = librosa.power_to_db(mel_og)

audio_butterworth = augmenter._add_frequency_mask(audio_check3)
mel_butterworth = librosa.feature.melspectrogram(y=audio_butterworth, sr=sr, n_mels=80)
log_mel_butterworth = librosa.power_to_db(mel_butterworth)

log_mel_specaugment = torch.tensor(log_mel_og)
freq_mask = T.FrequencyMasking(freq_mask_param=15)
log_mel_specaugment = freq_mask(torch.tensor(log_mel_og, dtype=torch.float32))

mask_value = torch.tensor(log_mel_og.min(), dtype=torch.float32)

log_mel_specaugment = torch.where(
    log_mel_specaugment == 0,
    torch.full_like(log_mel_specaugment, mask_value),
    log_mel_specaugment
).detach().cpu().numpy()

fig, ax = plt.subplots(1, 3, figsize=(18, 5))

librosa.display.specshow(log_mel_og, sr=sr, x_axis='time', y_axis='mel', ax=ax[0],cmap='viridis')
ax[0].set_title('Original')

librosa.display.specshow(log_mel_butterworth, sr=sr, x_axis='time', y_axis='mel', ax=ax[1],cmap='viridis')
ax[1].set_title('Butterworth Filter')

librosa.display.specshow(log_mel_specaugment, sr=sr, x_axis='time', y_axis='mel', ax=ax[2],cmap='viridis')
ax[2].set_title('SpecAugment Frequency Masking')

plt.tight_layout()
plt.show()

As visible from the last spectogram normal SpecAugment Frequency Masking is too crude and doesnt take into account phase relations (the perfect rectangular block )

In [ ]:
#Final spectogram between actual and random distortions
audio_check4, sr = librosa.load(sample_file, sr=16000)

mel_original = librosa.feature.melspectrogram(y=audio_check4, sr=sr, n_mels=80)
log_mel_original = librosa.power_to_db(mel_original)

audio_distorted = augmenter.augment(audio_check4)
if isinstance(audio_distorted, tf.Tensor):
    audio_distorted = audio_distorted.numpy()
mel_distorted = librosa.feature.melspectrogram(y=audio_distorted, sr=sr, n_mels=80)
log_mel_distorted = librosa.power_to_db(mel_distorted)

sample_output = "/content/drive/MyDrive/ASR/data/103-1241-0000_dist.flac"
sf.write(sample_output,audio_distorted,sr)
y, sr = librosa.load(sample_output, sr=16000)
Audio(y, rate=sr)
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

librosa.display.specshow(log_mel_original, sr=sr, x_axis='time', y_axis='mel', ax=ax[0],cmap='viridis')
ax[0].set_title('Original')
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Mel Frequency')

librosa.display.specshow(log_mel_distorted, sr=sr, x_axis='time', y_axis='mel', ax=ax[1],cmap='viridis')
ax[1].set_title('After Random Distortions')
ax[1].set_xlabel('Time')
ax[1].set_ylabel('Mel Frequency')

plt.tight_layout()
plt.show()


In [ ]:
Audio(audio_check4, rate=sr)

In [ ]:
aud_dist, sr = librosa.load(sample_output, sr=16000)
Audio(aud_dist, rate=sr)

In [ ]:
#Final spectogram between actual and random distortions
audio_check_all, sr = librosa.load(sample_file, sr=16000)

mel_og = librosa.feature.melspectrogram(y=audio_check_all, sr=sr, n_mels=80)
log_mel_og = librosa.power_to_db(mel_og)

audio_noise = augmenter._add_noise(audio_check_all)
audio_reverb = augmenter._add_reverb(audio_check_all)
audio_shuffle = augmenter._segment_shuffle(audio_check_all)
audio_stretch = augmenter._time_stretch(audio_check_all)
audio_gaps = augmenter._add_missing_gaps(audio_check_all)
audio_mask = augmenter._add_frequency_mask(audio_check_all)

mel_noise = librosa.feature.melspectrogram(y=audio_noise, sr=sr, n_mels=80)
log_mel_noise = librosa.power_to_db(mel_noise)

mel_reverb = librosa.feature.melspectrogram(y=audio_reverb, sr=sr, n_mels=80)
log_mel_reverb = librosa.power_to_db(mel_reverb)

mel_shuffle = librosa.feature.melspectrogram(y=audio_shuffle, sr=sr, n_mels=80)
log_mel_shuffle = librosa.power_to_db(mel_shuffle)

mel_stretch = librosa.feature.melspectrogram(y=audio_stretch, sr=sr, n_mels=80)
log_mel_stretch = librosa.power_to_db(mel_stretch)

mel_gaps = librosa.feature.melspectrogram(y=audio_gaps, sr=sr, n_mels=80)
log_mel_gaps = librosa.power_to_db(mel_gaps)

mel_mask = librosa.feature.melspectrogram(y=audio_mask, sr=sr, n_mels=80)
log_mel_mask = librosa.power_to_db(mel_mask)

fig, ax = plt.subplots(3, 3, figsize=(18, 18))

librosa.display.specshow(log_mel_og, sr=sr, x_axis='time', y_axis='mel', ax=ax[0][0],cmap='viridis')
ax[0][0].set_title('Original')
ax[0][1].axis('off')
ax[0][2].axis('off')

librosa.display.specshow(log_mel_noise, sr=sr, x_axis='time', y_axis='mel', ax=ax[1][0],cmap='viridis')
ax[1][0].set_title('Noise')

librosa.display.specshow(log_mel_reverb, sr=sr, x_axis='time', y_axis='mel', ax=ax[1][1],cmap='viridis')
ax[1][1].set_title('Reverb')

librosa.display.specshow(log_mel_shuffle, sr=sr, x_axis='time', y_axis='mel', ax=ax[1][2],cmap='viridis')
ax[1][2].set_title('Shuffle')

librosa.display.specshow(log_mel_stretch, sr=sr, x_axis='time', y_axis='mel', ax=ax[2][0],cmap='viridis')
ax[2][0].set_title('Time Stretch')

librosa.display.specshow(log_mel_gaps, sr=sr, x_axis='time', y_axis='mel', ax=ax[2][1],cmap='viridis')
ax[2][1].set_title('Missing Gaps')

librosa.display.specshow(log_mel_mask, sr=sr, x_axis='time', y_axis='mel', ax=ax[2][2],cmap='viridis')
ax[2][2].set_title('Freq Mask (Butterworth)')

for i in range(3):
    for j in range(3):
        ax[i][j].set_xlabel('Time')
        ax[i][j].set_ylabel('Mel Frequency')

plt.suptitle('Effect of Individual Augmentations on Log-Mel Spectrogram', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import time
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100")
sample_files = list(data_dir.rglob("*.flac"))[:100]
# sample_txt   = list(data_dir.rglob("*.txt"))[.sort]

start = time.time()

for i in range(len(sample_files)):
    audio_file = sample_files[i]
    log_mel = mel_log_gen(str(audio_file))

    if (i + 1) % 10 == 0:
        print(f"Processed {i+1}/{len(sample_files)} files")

end = time.time()

total_time = end - start
avg_time = total_time / len(sample_files)

print(f"\nTotal files: {len(sample_files)}")
print(f"Total time: {total_time:.2f} seconds")
print(f"Average per file: {avg_time:.3f} seconds")
print(f"Estimated for 28,539 files: {(avg_time * 28539) / 60:.1f} minutes")

In [9]:
import shutil
import os
input_root_dir = "/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/"

output_root_dir = "/content/drive/MyDrive/ASR/data/data_aug_transcriptions"
# ---
print(f"Starting transfer and conversion from '{input_root_dir}'...")
print(f"Output will be saved in '{output_root_dir}'.")
count=0
# Walk through the entire directory structure
for dirpath, _, filenames in os.walk(input_root_dir):
    for filename in filenames:
      if(not filename.endswith(".flac")):
        # Construct the full path to the source file
        count+=1
        if count%100==0:
          print(count)
        input_file_path = os.path.join(dirpath, filename)

        # Create the corresponding output directory structure
        relative_path = os.path.relpath(dirpath, input_root_dir)
        output_dir = os.path.join(output_root_dir, relative_path)
        os.makedirs(output_dir, exist_ok=True)
        output_file_path = os.path.join(output_dir, filename)
        shutil.copy2(input_file_path, output_file_path)
print("\n--- Process Complete ---")
print(f"New dataset is ready at: '{output_root_dir}'")

Starting transfer and conversion from '/content/drive/MyDrive/ASR/data/LibriSpeech/train-clean-100/'...
Output will be saved in '/content/drive/MyDrive/ASR/data/data_aug_transcriptions'.
100
200
300
400
500

--- Process Complete ---
New dataset is ready at: '/content/drive/MyDrive/ASR/data/data_aug_transcriptions'


In [11]:
print(count)

585
